In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/9/29 16:01
# @Author  : Wang Yujia
# @File    : basicinfo_outcomes.py
# @Description : 关于outcomes.tsv数据集的一些basic信息


[TOC]
# 0. what for
1. 旨在输出一些关于traces数据集的一些basic信息

# 1. Preparations
## 1.1 全局设置

In [9]:
# data path
outcomes_orignal_path = "../data/outcomes.tsv"
traces_original_path = "../data/traces.tsv"
# outcomes和traces共有的auction_id: `common_auction_id`
common_auction_id_path = "../data/common_auction_id.csv"

# 最终GT model需要的features（不包括np），其中`product_id` is not necessary
features_GT = ['auction_id','product_id','bidincrement','bidfee','retail']
# 衡量一场auction是否unique的标志
unique_setting = ['product_id', 'bidincrement', 'bidfee','retail']
# fixed-auction flag
flg_fixed = ['flg_fixedprice']

import pandas as pd
import numpy as np
import seaborn as sns

## 1.2 读取data

In [14]:
outcomes = pd.read_csv(outcomes_orignal_path, sep='\t')
traces = pd.read_csv(traces_original_path, sep='\t')
common_auction_id = pd.read_csv(common_auction_id_path, encoding="utf-8").auction_id


# 2. Info about outcomes.tsv
## 2.1 how many auctions are recorded

In [7]:
auction_id_unique = outcomes['auction_id'].unique()
n_auction_outcomes = auction_id_unique.shape[0]
print("outcomes.tsv记录了*{}*场auction".format(n_auction_outcomes))



outcomes.tsv记录了*121419*场auction


## 2.2 how many fixed-price auctions here?
1. recall that traces.tsv have no fixed-price auctions

In [15]:
#outcomes = outcomes[(features_GT+flg_fixed)]
idx = np.array(outcomes[flg_fixed] == 1)  # 记得转化成array，否则容易因为index不对应产生NaN值
outcomes_fixed = outcomes[idx]
print("outcomes.tsv 中一共有 *{}* 场auctions 是 fixed-price auction.".format(outcomes_fixed.shape[0]))

outcomes.tsv 中一共有 *2203* 场auctions 是 fixed-price auction.


## 2.3 Do all fixed-price('flg_fixedprice == 1') auctions have 'finalprice' == 0
1. 用来判断fixed-price的auction最后的成交价是不是都是0: *Yes*

In [18]:
print("There is totally *{}* fixed-price auctions whose 'finalprice' is NOT zero".format(sum(outcomes_fixed['finalprice'] == 0)))

There is totally *0* fixed-price auctions whose 'finalprice' is NOT zero
